# 4.2 模型参数的访问、初始化和共享

In [1]:
import torch
from torch import nn
from torch.nn import init

print(torch.__version__)

1.3.1


In [2]:
net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1))  # pytorch已进行默认初始化

print(net)
X = torch.rand(2, 4)
Y = net(X).sum()
print(X)
print(Y)
print(net(X))

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)
tensor([[2.1454e-01, 5.8480e-01, 2.6642e-01, 4.4525e-01],
        [9.9957e-05, 5.9802e-01, 4.2193e-01, 9.4472e-02]])
tensor(-1.4455, grad_fn=<SumBackward0>)
tensor([[-0.7347],
        [-0.7108]], grad_fn=<AddmmBackward>)


## 4.2.1 访问模型参数

In [6]:
print(type(net.named_parameters()))#
for name, param in net.named_parameters():
    print(name, param.size())
    # print(name,param.shape) 效果一致
    print(param)

<class 'generator'>
0.weight torch.Size([3, 4])
Parameter containing:
tensor([[-0.4227, -0.2299, -0.0784, -0.3677],
        [ 0.4695,  0.3014, -0.1142,  0.0521],
        [ 0.1549,  0.0847,  0.2217,  0.1758]], requires_grad=True)
0.bias torch.Size([3])
Parameter containing:
tensor([ 0.1046, -0.3073,  0.2669], requires_grad=True)
2.weight torch.Size([1, 3])
Parameter containing:
tensor([[-0.4575, -0.5627, -0.4043]], requires_grad=True)
2.bias torch.Size([1])
Parameter containing:
tensor([-0.5378], requires_grad=True)


In [4]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [ ]:
class MyModel(nn,Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        self.weight2 = torch.rand(20, 20)
    def forward(self, x):
        pass
    
n = MyModel()
for name, param in n.named_parameters():
    print(name)

In [5]:
weight_0 = list(net[0].parameters())[0]
print(list(net[0].parameters()))
print(weight_0.data)
print(weight_0.grad)
Y.backward()
print(weight_0.grad)

[Parameter containing:
tensor([[-0.4227, -0.2299, -0.0784, -0.3677],
        [ 0.4695,  0.3014, -0.1142,  0.0521],
        [ 0.1549,  0.0847,  0.2217,  0.1758]], requires_grad=True), Parameter containing:
tensor([ 0.1046, -0.3073,  0.2669], requires_grad=True)]
tensor([[-0.4227, -0.2299, -0.0784, -0.3677],
        [ 0.4695,  0.3014, -0.1142,  0.0521],
        [ 0.1549,  0.0847,  0.2217,  0.1758]])
None
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0868, -0.4782, -0.2783, -0.2182]])


## 4.2.2 初始化模型参数

In [ ]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

In [ ]:
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

## 4.2.3 自定义初始化方法

In [ ]:
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs() >= 5).float()

In [ ]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)

In [ ]:
for name, param in net.named_parameters():
    if 'bias' in name:
        param.data += 1
        print(name, param.data)

## 4.2.4 共享模型参数

In [ ]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear) 
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

In [ ]:
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

In [ ]:
x = torch.ones(1, 1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad)